In [1]:
#import libraries
import import_ipynb
import requests
import time 

#import ipynb files
from common.Extract_Twitter_Trends import Twitter_Trends
import config.Read_Configs as Read_Configs
from common.App import PubNub_Config
import common.My_Subscribe_Callback as My_Subscribe_Callback

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from C:\Users\tarvi\Twitter Trends Analysis\Phase 2\src\common\Extract_Twitter_Trends.ipynb
importing Jupyter notebook from C:\Users\tarvi\Twitter Trends Analysis\Phase 2\src\config\Read_Configs.ipynb
importing Jupyter notebook from C:\Users\tarvi\Twitter Trends Analysis\Phase 2\src\common\App.ipynb
importing Jupyter notebook from C:\Users\tarvi\Twitter Trends Analysis\Phase 2\src\common\My_Subscribe_Callback.ipynb


In [2]:
pubnub_configs = Read_Configs.read_pubnub_configurations()

C:\Users\tarvi\Twitter Trends Analysis\Phase 2\src\data\configuration.yaml


In [3]:
stream = PubNub_Config(pubnub_configs['subscribe_key'], pubnub_configs['publish_key'], pubnub_configs['user_id'] )

In [4]:
FirstSubscribeCallback = My_Subscribe_Callback.FirstSubscribeCallback()
stream.run(FirstSubscribeCallback)
time.sleep(30)

15 2023-02-09


In [5]:
hour = My_Subscribe_Callback.get_hour
date = My_Subscribe_Callback.get_date
print(hour, date)

15 2023-02-09


In [6]:
# URL for retreiving Twitter Trends configurations from API 
URL = "https://9188-2a02-a210-2ec2-1480-58ec-cacd-5b36-a6cf.ngrok-free.app/trend_read_configurations/"
  
# sending get request and saving the response 
r = requests.get(url = URL)

# populating config variables 
trends = r.json()[0]
country = r.json()[1]

In [7]:
trends = Twitter_Trends(country, date, hour, trends)
top_trends = trends.get_top_trends()
print (top_trends)

https://getdaytrends.com/netherlands/2023-02-09/15/
{1: '#feynec', 2: 'Unilever', 3: 'Emine'}
